# Coronavirus Pandemic (COVID-19)

- Coronavirus disease (COVID-19) is an infectious disease caused by the SARS-CoV-2 virus.
- Most people who fall sick with COVID-19 will experience mild to moderate symptoms and recover without special treatment. However, some will become seriously ill and require medical attention.

Learn more about Coronavirus disease (COVID-19) pandemic: [WHO](https://www.who.int/emergencies/diseases/novel-coronavirus-2019)

COVID Live - Coronavirus Statistics: [Worldometers](https://www.worldometers.info/coronavirus/)


# Import Libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time

from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

import datetime
import operator  # exports a set of efficient functions corresponding to the intrinsic operators of Python
plt.style.use('seaborn-poster')

%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
